In [4]:
# Preprocessing
# This is needed only once
import matplotlib.pylab as plt
import numpy as np
# Saving input images in frames.npy and groundtruth images in frames_gt.npy
frames = []
frames_gt = []

for k in range(1099):
    imageData = plt.imread("input/img"+str(k+1)+".jpg")
    frames.append(imageData)
    imageData_gt = plt.imread("groundtruth/gt"+str(k+1)+".png")
    frames_gt.append(imageData_gt)

frames = np.array(frames)
np.save('frames',frames)    
frames_gt = np.array(frames_gt)
np.save('frames_gt',frames_gt)

In [5]:
import matplotlib.pylab as plt
import numpy as np
from scipy import ndimage
x1 = np.load('frames.npy')
x1 = x1[0:200]

from skimage.color import rgb2gray
frames_gt = np.load('frames_gt.npy')
frames_gt = rgb2gray(frames_gt);

In [6]:
x8 = np.zeros([200,240,360,3])
# Get the 8x8 frame by convolving

In [7]:
for i in range(200):
    for j in range(3):
        x8[i,:,:,j] = ndimage.convolve(x1[i,:,:,j],1/64*np.ones([8,8]),mode = 'nearest')

In [8]:
#Getting the  ycbcr for both x1 and x8
y1 = np.asarray(16 + (65.738*x1[:,:,:,0]+129.057*x1[:,:,:,1]+25.064*x1[:,:,:,2])/256)
cb1 = np.asarray(128 + (-37.945*x1[:,:,:,0]-74.494*x1[:,:,:,1]+112.439*x1[:,:,:,2])/256)
cr1 = np.asarray(128 + (112.439*x1[:,:,:,0] - 94.154*x1[:,:,:,1] - 18.285*x1[:,:,:,2])/256)


y8 = np.asarray(16 + (65.738*x8[:,:,:,0]+129.057*x8[:,:,:,1]+25.064*x8[:,:,:,2])/256)
cb8 = np.asarray(128 + (-37.945*x8[:,:,:,0]-74.494*x8[:,:,:,1]+112.439*x8[:,:,:,2])/256)
cr8 = np.asarray(128 + (112.439*x8[:,:,:,0] - 94.154*x8[:,:,:,1] - 18.285*x8[:,:,:,2])/256)

In [9]:
#subtract these mean values(already stored)
#update the mean values
y1m = np.mean(y1[0:200],axis = 0)
y1 = y1 - y1m
y8m = np.mean(y8[0:200],axis = 0)
y8 = y8 - y8m
cb1m = np.mean(cb1[0:200],axis = 0)
cb1 = cb1 - cb1m
cb8m = np.mean(cb8[0:200],axis = 0)
cb8 = cb8 - cb8m
cr1m = np.mean(cr1[0:200],axis = 0)
cr1 = cr1 - cr1m
cr8m = np.mean(cr8[0:200],axis = 0)
cr8 = cr8 - cr8m

x1rm = np.mean(x1[0:200,:,:,0],axis = 0)
x1[:,:,:,0] = x1[:,:,:,0] - x1rm
x8rm = np.mean(x8[0:200,:,:,0],axis = 0)
x8[:,:,:,0] = x8[:,:,:,0] - x8rm
x1gm = np.mean(x1[0:200,:,:,1],axis = 0)
x1[:,:,:,1] = x1[:,:,:,1] - x1gm
x8gm = np.mean(x8[0:200,:,:,1],axis = 0)
x8[:,:,:,1] = x8[:,:,:,1] - x8gm
x1bm = np.mean(x1[0:200,:,:,2],axis = 0)
x1[:,:,:,2] = x1[:,:,:,2] - x1bm
x8bm = np.mean(x8[0:200,:,:,2],axis = 0)
x8[:,:,:,2] = x8[:,:,:,2] - x8bm

In [10]:
#create new variables x1rgb and x8rgb and x1ycbcr and x8ycbcr
x1rgb = np.square(x1[:,:,:,0])+np.square(x1[:,:,:,1])+np.square(x1[:,:,:,2])
x8rgb = np.square(x8[:,:,:,0])+np.square(x8[:,:,:,1])+np.square(x8[:,:,:,2])

x1ycbcr = np.square(y1)+np.square(cb1)+np.square(cr1)
x8ycbcr = np.square(y8)+np.square(cb8)+np.square(cr8)

In [11]:
#update x1ycbcr and x8ycbcr and t1ycbcr and t2 ycbcr
v1 = np.mean(x1ycbcr[0:200],axis = 0)
v8 = np.mean(x8ycbcr[0:200],axis = 0)


x1ycbcr = np.square(x1ycbcr-v1)
x8ycbcr = np.square(x8ycbcr-v8)

In [12]:
#new variables ycbcr and rgb
ycbcr = np.where(x1ycbcr<x8ycbcr,x1ycbcr,x8ycbcr)
rgb = np.where(x1rgb<x8rgb,x1rgb,x8rgb)

In [13]:
#some more thresholds; ty and tr; need update
t1 = np.mean(x1ycbcr,axis = 0)
t8 = np.mean(x8ycbcr,axis = 0)
ty = np.where(t1>t8,t1,t8)


t1_ = np.mean(x1rgb,axis = 0)
t8_ = np.mean(x8rgb,axis = 0)
tr = np.where(t1_>t8_,t1_,t8_)

In [14]:
#final comparison bw ycbcr and ty, rgb and tr
outputy = np.where(ycbcr>ty,255,0)
outputr = np.where(rgb>tr,255,0)

In [ ]:
for i in range(200):
    plt.imshow(np.where(outputr[i]>0,0,outputy[i]))
    plt.pause(0.000000001)
    print(i)

In [ ]:
frames = np.load('frames.npy')
alpha = 0.001
#alpha is the tuning parameter; alpha is inversely proportional to frame rate(fps)
for i in range(200,1099):
  x1 = frames[i]
  x8 = np.zeros([240,360,3])
  for j in range(3):
    x8[:,:,j] = ndimage.convolve(x1[:,:,j],1/64*np.ones([8,8]),mode = 'nearest')
  y1 = np.asarray(16 + (65.738*x1[:,:,0]+129.057*x1[:,:,1]+25.064*x1[:,:,2])/256)
  cb1 = np.asarray(128 + (-37.945*x1[:,:,0]-74.494*x1[:,:,1]+112.439*x1[:,:,2])/256)
  cr1 = np.asarray(128 + (112.439*x1[:,:,0] - 94.154*x1[:,:,1] - 18.285*x1[:,:,2])/256)
  y8 = np.asarray(16 + (65.738*x8[:,:,0]+129.057*x8[:,:,1]+25.064*x8[:,:,2])/256)
  cb8 = np.asarray(128 + (-37.945*x8[:,:,0]-74.494*x8[:,:,1]+112.439*x8[:,:,2])/256)
  cr8 = np.asarray(128 + (112.439*x8[:,:,0] - 94.154*x8[:,:,1] - 18.285*x8[:,:,2])/256)
  y1m = (y1m*(1-alpha)) +(y1*alpha)
  cb1m = (cb1m*(1-alpha)) +(cb1*alpha)
  cr1m = (cr1m*(1-alpha)) +(cr1*alpha)
  y8m = (y8m*(1-alpha)) +(y8*alpha)
  cb8m = (cb8m*(1-alpha)) +(cb8*alpha)
  cr8m = (cr8m*(1-alpha)) +(cr8*alpha)
  y1 = y1 - y1m
  y8 = y8 - y8m
  cb1 = cb1 - cb1m
  cb8 = cb8 - cb8m
  cr1 = cr1 - cr1m
  cr8 = cr8 - cr8m

  x1rm = (x1rm*(1-alpha)) +(x1[:,:,0]*alpha)
  x1gm = (x1gm*(1-alpha)) +(x1[:,:,1]*alpha)
  x1bm = (x1bm*(1-alpha)) +(x1[:,:,2]*alpha)
  x8rm = (x8rm*(1-alpha)) +(x8[:,:,0]*alpha)
  x8gm = (x8gm*(1-alpha)) +(x8[:,:,1]*alpha)
  x8bm = (x8bm*(1-alpha)) +(x8[:,:,2]*alpha)
  x1[:,:,0] = x1[:,:,0] - x1rm
  x8[:,:,0] = x8[:,:,0] - x8rm
  x1[:,:,1] = x1[:,:,1] - x1gm
  x8[:,:,1] = x8[:,:,1] - x8gm
  x1[:,:,2] = x1[:,:,2] - x1bm
  x8[:,:,2] = x8[:,:,2] - x8bm

  x1rgb = np.square(x1[:,:,0])+np.square(x1[:,:,1])+np.square(x1[:,:,2])
  x8rgb = np.square(x8[:,:,0])+np.square(x8[:,:,1])+np.square(x8[:,:,2])
  x1ycbcr = np.square(y1)+np.square(cb1)+np.square(cr1)
  x8ycbcr = np.square(y8)+np.square(cb8)+np.square(cr8)

  v1 = (v1*(1-alpha))+ (x1ycbcr*alpha)
  v8 = (v8*(1-alpha))+ (x8ycbcr*alpha)


  x1ycbcr = np.square(x1ycbcr-v1)
  x8ycbcr = np.square(x8ycbcr-v8)


  ycbcr = np.where(x1ycbcr<x8ycbcr,x1ycbcr,x8ycbcr)
  rgb = np.where(x1rgb<x8rgb,x1rgb,x8rgb)

  t1 = (t1*(1-alpha))+(x1ycbcr*alpha)
  t8 = (t8*(1-alpha))+(x8ycbcr*alpha)
  t1_ = (t1_*(1-alpha))+(x1rgb*alpha)
  t8_ = (t8_*(1-alpha))+(x8rgb*alpha)
  ty = np.where(t1<t8,t8,t1)
  tr = np.where(t1_<t8_,t8_,t1_)
  # ty = (ty*(1-alpha))+(np.where(x1ycbcr<x8ycbcr,x8ycbcr,x1ycbcr)*alpha)
  # tr = (tr*(1-alpha))+(np.where(x1rgb<x8rgb,x8rgb,x1rgb)*alpha) 


  outputy = np.where(ycbcr>ty,255,0)
  outputr = np.where(rgb>tr,255,0)
  output = np.where(outputr>0,0,outputy)
  plt.imshow(output)
  gt = frames_gt[i]

  tp = np.where(output==255, 1,0)
  tp = np.where(gt>0.9,tp,0)
  tn = np.where(output==0, 1,0)
  tn = np.where(gt<0.1,tn,0)
  fp = np.where(output==255, 1,0)
  fp = np.where(gt<0.1,fp,0)
  fn = np.where(output==0, 1,0)
  fn = np.where(gt>0.9,fn,0)

  Recall  = np.sum(np.sum(tp))/(np.sum(np.sum(tp))+np.sum(np.sum(fn)))
  Precision = np.sum(np.sum(tp))/(np.sum(np.sum(tp))+np.sum(np.sum(fp)))
  F1 = (2*Recall*Precision)/(Precision+Recall)
  print("Recall :" , Recall)
  print("Precision :" , Precision)
  print("F1 Score :" , F1)
  plt.pause(0.000000001)
  plt.imshow(gt)
  plt.pause(0.000000001)
  print(i)